# HW-SW Cosimulation

- Colab이 '클라우드 기반의 격리된 환경'이라는 점 떄문에 이번 실습을 바로 실행하기는 어렵다. 따라서, 교재에서 가이드하는 것처럼 linux emulator 를 설치하고 linux 상에서 실행하기로 한다.
  - GUI 창(xterm, gnome-terminal)을 띄울 수 없다.
  - Colab 내부에서도 프로세스 간 통신은 가능하지만, 두 개의 창을 동시에 띄워놓고 한쪽에서는 C 코드가 돌아가고, 다른 쪽에서는 iverilog가 파형을 만드는 모습을 실시간으로 모니터링하기가 매우 어렵다.
  - GTKWave 파형 분석의 불편함.

# Tools 설치 및 파일 Upload

## WSL(Windows Subsystem for Linux) 설치 (in your local PC)

- 사전 확인
  - (Windows 기능에서 "가상 머신 플랫폼"을 켜고, ② BIOS에서 CPU 가상화(VT-x/AMD-V) 를 활성화해야 함.)
  - 'Windows 기능 켜기/끄기' 에서 다음 항목 체크
     - 가상 머신 플랫폼 (Virtual Machine Platform)
     - Windows 하이퍼바이저 플랫폼 (Windows Hypervisor Platform) (권장)
     - Linux용 Windows 하위 시스템
  - BIOS 에서 "Intel Virtualization" enabled 되어 있어야 함.
- 설치 과정
  - powershell open (또는 Window note)
  - wsl --list --online (설치 가능한 배포판 목록 확인)
  - wsl --install -d Ubuntu-24.04 (특정 배포판 설치)
  - wsl -l -v (이미 설치된 배포판 확인)


## Install iverilog
  - 최신 버전을 사용하기 위해 직접 빌드해서 설치한다.

```
[wsl 에서 iverilog 설치]

% sudo apt update            // apt update
% sudo apt install -y autoconf gperf make gcc g++ bison flex
% cd ~rtajeong (username)
% git clone https://github.com/steveicarus/iverilog
  (permission 문제 때문에 wsl 의 home directory 에서 설치하여야 함)
% cd iverilog
% sh autoconf.sh
% ./configure
% make
% sudo make install
% iverilog -h
% (사용 예) iverilog -o hello_world hello_world.sv
```

## GTKwave 설치

```
% sudo apt install gtkwave
% (사용 예) gtkwave wave.vcd
```

## Gnome (GNU Network Object Model Environment) 설치
  - 역할: 리눅스 커널 위에서 돌아가는 그래픽 터미널

```
% sudo apt update
% sudo apt install gnome-terminal -y
% (사용할 때에는) gnome-terminal &
```

# Exercise

## IPC 실습

- top.v: led_val이라는 8비트 레지스터를 가지고 있는 모듈

In [1]:
%%writefile top.v
module top (
    input clk,
    input [7:0] data_in,
    input write_en,
    output reg [7:0] led_val
);
    always @(posedge clk) begin
        if (write_en) led_val <= data_in;
    end
endmodule

Writing top.v


- C++ 시뮬레이터 및 소켓 클라이언트 (sim_main.cpp)
  - 이 코드는 시뮬레이터이면서 동시에 소켓 클라이언트 역할을 한다. 서버로부터 데이터를 받아 하드웨어 신호를 흔든다.

In [2]:
%%writefile sim_main.cpp
#include <iostream>
#include <sys/socket.h>
#include <arpa/inet.h>
#include <unistd.h>
#include "Vtop.h"
#include "verilated.h"

int main(int argc, char** argv) {
    Verilated::commandArgs(argc, argv);
    Vtop* top = new Vtop; // 하드웨어 인스턴스

    // 소켓 설정
    int sock = socket(AF_INET, SOCK_STREAM, 0);
    struct sockaddr_in serv_addr;
    serv_addr.sin_family = AF_INET;
    serv_addr.sin_port = htons(9999);
    inet_pton(AF_INET, "127.0.0.1", &serv_addr.sin_addr);

    std::cout << "[Simulator] 서버 접속 시도 중..." << std::endl;
    while (connect(sock, (struct sockaddr*)&serv_addr, sizeof(serv_addr)) < 0) {
        usleep(100000); // 서버가 뜰 때까지 대기
    }
    std::cout << "[Simulator] 서버 접속 완료!" << std::endl;

    while (!Verilated::gotFinish()) {
        char buffer[1024] = {0};
        int valread = read(sock, buffer, 1024);
        if (valread <= 0) break;

        // 받은 데이터를 하드웨어 신호로 주입 (예: 받은 숫자를 led_val에 씀)
        int received_val = atoi(buffer);
        top->data_in = received_val;
        top->write_en = 1;

        // 클럭 한 주기 발생
        top->clk = 0; top->eval();
        top->clk = 1; top->eval();
        top->write_en = 0;

        std::cout << "[Simulator] HW 레지스터 업데이트: " << (int)top->led_val << std::endl;

        if (received_val == 255) break; // 종료 조건
    }

    delete top;
    close(sock);
    return 0;
}

Writing sim_main.cpp


- 시뮬레이션 실행 (Python 서버와 동시 실행)
  - Python으로 서버(명령을 내리는 프로그램)를 만들고, 위에서 만든 시뮬레이터를 컴파일해서 동시에 돌린다.
  - Why C 대신 Python?
    - 코드의 직관성: C로 소켓 서버를 만들려면 struct sockaddr_in, bind, listen, accept 등 수십 줄의 복잡한 코드가 필요하다. 반면 Python은 단 몇 줄로 "메시지 패싱의 흐름" 그 자체를 보여주기에 가장 좋다.
    - 유연한 문자열 처리: 소켓을 통해 "10", "128" 같은 숫자를 보낼 때, C는 버퍼 관리와 타입 변환이 까다롭지만 Python은 매우 자유롭다.
    - 프로세스 제어: Colab 환경에서 C로 만든 두 프로그램을 동시에 띄우고 관리하는 것보다, Python의 multiprocessing 라이브러리를 쓰는 것이 실행 흐름을 제어하기 훨씬 안전하고 쉽다.

In [ ]:
import os
import multiprocessing
import socket
import time

# 1. 시뮬레이터 컴파일 (Verilator)
def build_sim():
    print("컴파일 중...")
    os.system("verilator -Wall --cc top.v --exe sim_main.cpp")
    os.system("make -C obj_dir -f Vtop.mk Vtop")
    print("컴파일 완료.")

# 2. 명령을 내리는 서버 (SW 계층 비유)
def command_server():
    server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    server.bind(('127.0.0.1', 9999))
    server.listen(1)

    conn, _ = server.accept()
    print("[Server] 명령 전송 시작...")

    for val in ["10", "50", "128", "255"]: # 하드웨어에 보낼 값들
        time.sleep(1)
        conn.send(val.encode())
        print(f"[Server] HW로 데이터 {val} 전송함")

    conn.close()
    server.close()

# 3. 전체 실행
if __name__ == "__main__":
    build_sim()

    # 서버 프로세스 시작
    srv = multiprocessing.Process(target=command_server)
    srv.start()

    # 시뮬레이터 실행 (HW 계층 비유)
    time.sleep(2) # 서버가 바인딩할 시간 확보
    print("[System] 시뮬레이터 가동")
    os.system("./obj_dir/Vtop")

    srv.join()
    print("[System] 모든 테스트 종료")

컴파일 중...
컴파일 완료.
[System] 시뮬레이터 가동


- 정리
  - 이 예제에서 각 역할은 다음과 같이 매칭된다.
  - top.v: 우리가 검증하려는 하드웨어(UART 등) 역할.
  - sim_main.cpp: 하드웨어를 감싸고 있는 BFM 역할이다. 소켓에서 데이터를 받아 하드웨어 핀(top->data_in)을 실제로 흔든다.
  - command_server (Python): 사용자의 C 루틴(main.c) 역할이다. 소켓을 통해 "이 값을 하드웨어에 써라"라고 명령을 던진다.

-----------------------